In [1]:
import datetime
from cryptography.hazmat.primitives import serialization
import requests
import pandas as pd

In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
HTTPResponse = requests.packages.urllib3.response.HTTPResponse
orig_HTTPResponse__init__ = HTTPResponse.__init__
def new_HTTPResponse__init__(self, *args, **kwargs):
    orig_HTTPResponse__init__(self, *args, **kwargs)
    try:
        self.peer_certificate = self._connection.peer_certificate
    except AttributeError:
        pass
HTTPResponse.__init__ = new_HTTPResponse__init__

HTTPAdapter = requests.adapters.HTTPAdapter
orig_HTTPAdapter_build_response = HTTPAdapter.build_response
def new_HTTPAdapter_build_response(self, request, resp):
    response = orig_HTTPAdapter_build_response(self, request, resp)
    try:
        response.peer_certificate = resp.peer_certificate
    except AttributeError:
        pass
    return response
HTTPAdapter.build_response = new_HTTPAdapter_build_response

HTTPSConnection = requests.packages.urllib3.connection.HTTPSConnection
orig_HTTPSConnection_connect = HTTPSConnection.connect
def new_HTTPSConnection_connect(self):
    orig_HTTPSConnection_connect(self)
    try:
        self.peer_certificate = self.sock.connection.get_peer_certificate()
    except AttributeError:
        pass
HTTPSConnection.connect = new_HTTPSConnection_connect

In [21]:
def check_certificate(target, method=requests.head, verify=False, timeout=20.):
    tic = datetime.datetime.utcnow()
    response = method(target, timeout=timeout, verify=verify)
    tac = datetime.datetime.utcnow()
    print(response, target)
    certificate = response.peer_certificate
    result = {
        'request': {
            'target': target,
            'method': method.__name__.upper(),
            'verify': verify,
            'timeout': timeout,
            'timestamp': tic,
            'elapsed': (tac - tic)/datetime.timedelta(seconds=1),
            'status': response.status_code,
            'headers': dict(response.headers)
        },
        'certificate': {
            'subject': {k.decode(): v.decode() for k, v in dict(certificate.get_subject().get_components()).items()},
            'issuer': {k.decode(): v.decode() for k, v in dict(certificate.get_issuer().get_components()).items()},
            'serial': certificate.get_serial_number().to_bytes(20, byteorder='big').hex(),
            'version': certificate.get_version(),
            'algorithm': certificate.get_signature_algorithm().decode(),
            'key_size': certificate.get_pubkey().bits(),
            'key_type': certificate.get_pubkey().type(),
            'digest': certificate.digest("sha1").decode(),
            'start': datetime.datetime.strptime(certificate.get_notBefore().decode(), '%Y%m%d%H%M%SZ'),
            'stop': datetime.datetime.strptime(certificate.get_notAfter().decode(), '%Y%m%d%H%M%SZ'),
            'expired': certificate.has_expired(),
            'pem': certificate.to_cryptography().public_bytes(encoding=serialization.Encoding.PEM).decode()
        }
    }
    result["certificate"]["expires"] = (result["certificate"]["stop"] - result["request"]["timestamp"])/datetime.timedelta(days=1)
    return result

In [22]:
targets = [
    "https://ulb.be",
    "https://auth.ulb.be",
    "https://bib.ulb.be",
    "https://bib-ulb-be.ezproxy.ulb.ac.be/",
    "https://sso-cas.ulb.ac.be",
    "https://cibleplus.ulb.ac.be",
    "https://gehol.ulb.ac.be",
    "https://www.pub-ulb.be/",
    "https://cirem.ulb.be/",
    "https://fari.brussels",
    "https://energy.sc.ulb.ac.be/grafana",
]

In [23]:
results = []
for target in targets:
    result = check_certificate(target)
    results.append(result)

<Response [301]> https://ulb.be
<Response [302]> https://auth.ulb.be
<Response [200]> https://bib.ulb.be
<Response [302]> https://bib-ulb-be.ezproxy.ulb.ac.be/
<Response [404]> https://sso-cas.ulb.ac.be
<Response [302]> https://cibleplus.ulb.ac.be
<Response [200]> https://gehol.ulb.ac.be
<Response [200]> https://www.pub-ulb.be/
<Response [200]> https://cirem.ulb.be/
<Response [200]> https://fari.brussels


ConnectionError: HTTPSConnectionPool(host='energy.sc.ulb.ac.be', port=443): Max retries exceeded with url: /grafana (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8c6a9aeac8>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [24]:
df = pd.json_normalize(results)
df.columns = [column.replace('.', '_').lower() for column in df.columns]
df

,request_target,request_method,request_verify,request_timeout,request_timestamp,request_elapsed,request_status,request_headers_content-length,request_headers_location,certificate_subject_c,...,request_headers_content-encoding,request_headers_keep-alive,request_headers_x-powered-by,request_headers_x-pingback,request_headers_link,request_headers_via,request_headers_x-cache-hits,request_headers_x-cache,request_headers_accept-ranges,request_headers_age
0,https://ulb.be,HEAD,False,20.0,2021-05-16 22:07:54.604762,0.099473,301,0,https://www.ulb.be/,BE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://auth.ulb.be,HEAD,False,20.0,2021-05-16 22:07:54.705792,0.251647,302,NaN,/login,BE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://bib.ulb.be,HEAD,False,20.0,2021-05-16 22:07:54.958627,0.520211,200,NaN,NaN,BE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://bib-ulb-be.ezproxy.ulb.ac.be/,HEAD,False,20.0,2021-05-16 22:07:55.480034,0.126404,302,NaN,https://login.ezproxy.ulb.ac.be/login?qurl=htt...,BE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://sso-cas.ulb.ac.be,HEAD,False,20.0,2021-05-16 22:07:55.607858,0.258133,404,NaN,NaN,BE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://cibleplus.ulb.ac.be,HEAD,False,20.0,2021-05-16 22:07:55.867237,0.148407,302,NaN,mng/login,BE,...,gzip,timeout=20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://gehol.ulb.ac.be,HEAD,False,20.0,2021-05-16 22:07:56.016955,0.245976,200,NaN,NaN,BE,...,NaN,NaN,"PHP/5.6.9, ASP.NET",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.pub-ulb.be/,HEAD,False,20.0,2021-05-16 22:07:56.264766,0.107500,200,NaN,NaN,NaN,...,NaN,"timeout=5, max=100",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,https://cirem.ulb.be/,HEAD,False,20.0,2021-05-16 22:07:56.373427,1.522999,200,NaN,NaN,BE,...,NaN,"timeout=5, max=100",NaN,https://cirem.ulb.be/xmlrpc.php,"<https://cirem.ulb.be/wp-json/>; rel=""https://...",NaN,NaN,NaN,NaN,NaN
9,https://fari.brussels,HEAD,False,20.0,2021-05-16 22:07:57.898258,2.170444,200,NaN,NaN,NaN,...,NaN,NaN,PHP/7.4.5,NaN,"<https://fari.brussels/wp-json/>; rel=""https:/...","1.1 varnish (Varnish/6.3), 1.1 varnish (Varnis...",0,MISS,bytes,0
